In [87]:
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce, cmp_to_key
from itertools import groupby, combinations, permutations
from time import time
from queue import PriorityQueue
from math import prod, ceil, comb, perm
from collections import defaultdict
import string


def print_mat(mat):
    for row in mat:
        print(''.join(row))


files = [f for f in listdir('.') if isfile(join('.', f))]
pat = re.compile('(ex(\d)+.txt)|(in(\d)*.txt)')
files = [f for f in files if pat.match(f)]

box = widgets.Select(
    options=files,
    value='ex1.txt',
    rows=4,
    description='Input file:',
    disabled=False
)

display(box)


Select(description='Input file:', index=1, options=('in.txt', 'ex1.txt'), rows=4, value='ex1.txt')

## Parse file

In [91]:
lines = list(map(lambda l: l.rstrip(), open(box.value, 'r').readlines()))

label = re.compile('([A-Z][A-Z])')
rate = re.compile('rate=(\d+)')
labels  = [label.findall(line) for line in lines]
rates = [int(rate.search(line).groups()[0]) for line in lines]
V = {l[0]: l[1:] for l in labels}
R = {l[0]: r for l, r in zip(labels, rates)}

print("Done parsing \"{name}\"".format(name=box.value))


Done parsing "in.txt"


### Part 1

In [92]:
def dist(a):
    Q = [a]
    D = {a: 0}
    while Q:
        x = Q.pop(0)
        for y in V[x]:
            if y not in D:
                D[y] = D[x] + 1
                Q.append(y)
    return D


def heuristic(p, t, Y):
    r = np.array(sorted(map(R.get, Y), reverse=True))
    T = np.full(len(r), t) - np.insert(np.cumsum(np.full(len(r)-1, 2)), 0, 0)
    T[T < 0] = 0
    return np.sum(-r * T) + p


D = {}
for x in V:
    for y, d in dist(x).items():
        D[(x, y)] = d
    D[("", x)] = 0
    D[(x, "")] = 0


remaining = [v for v in V if R[v] > 0]
Q = PriorityQueue()
Q.put((0, (30, 'AA', remaining)))
minp = 0
visited = set()

while not Q.empty():
    p, state = Q.get()
    t, X, Y = state
    visited.add(X)
    minp = min(minp, p)
    if t <= 0 or len(Y) == 0:
        continue
    h = heuristic(p, t, Y)
    if h <= minp:
        for i in range(len(Y)):
            y = Y[i]
            if R[y] == 0:
                continue
            if X + y not in visited:
                nt = t - D[(X[-2:], y)] - 1
                Q.put((p - R[y] * nt, (nt, X + y, Y[:i] + Y[i+1:])))

pprint(abs(minp))


1792


### Part 2

In [93]:
T = {}
remaining = [v for v in V if R[v] > 0]
Q = PriorityQueue()
Q.put((0, (26, 'AA', remaining)))
minp = 0
visited = set()

while not Q.empty():
    p, state = Q.get()
    t, X, Y = state
    path = set(list(zip(X[::2], X[1::2])))
    T[X] = (abs(p), path)
    visited.add(X)
    minp = min(minp, p)
    if t <= 0 or len(Y) == 0:
        continue
    h = heuristic(p, t, Y)
    if h <= minp:
        for i in range(len(Y)):
            y = Y[i]
            if R[y] == 0:
                continue
            if X + y not in visited:
                nt = t - D[(X[-2:], y)] - 1
                Q.put((p - R[y] * nt, (nt, X + y, Y[:i] + Y[i+1:])))


T = sorted(list(T.values()), key=lambda t: t[0])[::-1]

best = 0
for (x, p1) in T:
    for (y, p2) in T:
        if len(p1.intersection(p2)) > 1:
            continue

        p = x + y
        best = max(best, p)
        break

pprint(best)


2587
